# Praca domowa nr 2
## Kodowanie zmiennych kategorycznych
Michał Stawikowski

In [219]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate,  train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,  AdaBoostClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
import warnings
warnings.simplefilter('ignore')
df = pd.read_csv("CompassData.csv")
df = df.dropna()

In [213]:
# Przygotowujemy i dzielimy dane

X = df.drop(["is_recid", "first", "last", "dob", "c_jail_in", "c_jail_out"], axis = 1)
y = df.loc[:, 'is_recid']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2, random_state=21)
X_train = X_train.reset_index()
X_test = X_test.reset_index()
y_test = y_test.reset_index()
y_train = y_train.reset_index()
y_train = y_train.drop("index",axis = 1)
y_test = y_test.drop("index", axis = 1)
y = y.reset_index()
y = y.drop("index", axis = 1)
X = X.reset_index()
X = X.drop("index", axis = 1)
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9895 entries, 0 to 9894
Data columns (total 15 columns):
Unnamed: 0         9895 non-null int64
custody_count      9895 non-null int64
sex                9895 non-null object
age                9895 non-null int64
age_cat            9895 non-null object
race               9895 non-null object
juv_fel_count      9895 non-null int64
juv_misd_count     9895 non-null int64
juv_other_count    9895 non-null int64
priors_count       9895 non-null int64
c_charge_degree    9895 non-null object
c_charge_desc      9895 non-null object
bust_date          9895 non-null object
jail_time          9895 non-null float64
custody_sum        9895 non-null float64
dtypes: float64(2), int64(7), object(6)
memory usage: 1.1+ MB


Przygotowaliśmy i wczytaliśmy nasze dane z programu `Compass`, jak widać w ramce danych występuje kilka typów `object`, które nie nadają się do algorytmów klasyfikacji w `sklearn`. W celu umożliwienia stworzenia modelu możemy albo porzucić zmienne nienumeryczne albo spróbować je zakodować. Istnieją różne sposoby kodowania zmiennych kategorycznych. My użyjemy dwóch reprezentanów 'klasycznego kodowania':

* `OneHot Encoding`
* `Hashing Encoding`

oraz dwóch reprezentantów 'Bayesian Encoders', czyli:

* `Target Encoding`
* `LeaveOneOut Encoding`

Pierwsza grupa działa raczej opierając się na prostych zasadach przyporzodkowując unikalne wartości numeryczne kategorycznym zmiennym. Druga grupa używa informacji ze zmiennej zależnej i jest użyteczna w przypadku, gdy nasze zmienne posiadają zbyt dużą liczbę unikalnych poziomów.

Niestety klasyfikatory z pakietu `sklearn` nie radzą sobie z żadnymi zmiennymi kategorycznymi, więc będziemy musieli wszytskie kolumny zawierające takie wartości ręcznie poddać kodowaniu. 

#  Transformacja danych

In [192]:
# Tworzymy dwa encodery do porówynania
np.random.seed(42)
enc_Target = ce.TargetEncoder().fit(X_train, y_train)
enc_OneHot = ce.OneHotEncoder().fit(X_train, y_train)
enc_LeaveOneOut = ce.LeaveOneOutEncoder().fit(X_train, y_train)
enc_Hashing = ce.HashingEncoder().fit(X_train, y_train)

# Za ich pomocą transformujemy dane

training_Target = enc_Target.transform(X_train, y_train)
testing_Target = enc_Target.transform(X_test)

training_OneHot = enc_OneHot.transform(X_train)
testing_OneHot = enc_OneHot.transform(X_test)

training_LeaveOneOut = enc_LeaveOneOut.transform(X_train, y_train)
testing_LeaveOneOut = enc_LeaveOneOut.transform(X_test)

training_Hashing = enc_Hashing.transform(X_train)
testing_Hashing = enc_Hashing.transform(X_test)


Stworzyliśmy kodowania i przetransformowaliśmy dane. Jak widać na poniższysz tabelach, kodowania z drugiej grupy pewne wartości
numeryczne opisująca zależności jakie występowały między zmienną kategoryczną, a celem naszego przywidywania. Kodowanie `OneHot` dodało bardzo dużą ilość nowych kolumn kodujących zmienne kategoryczne, co prawdopodobnie nie wpłynie dobrze na skuteczność klasyfikacji.

In [123]:
# Porównanie wyników encodowania
X.head()

,Unnamed: 0,custody_count,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc,bust_date,jail_time,custody_sum
0,1,1,Female,23,Less than 25,African-American,0,0,0,0,(F3),Driving While License Revoked,2013-04-23,1.0,23.0
1,2,1,Female,21,Less than 25,Caucasian,0,0,0,0,(F2),Burglary Unoccupied Dwelling,2013-11-03,2.0,2.0
2,3,2,Male,27,25 - 45,Hispanic,0,0,1,18,(F3),Grand Theft in the 3rd Degree,2013-11-20,55.0,565.0
3,5,1,Male,41,25 - 45,Caucasian,0,0,0,11,(F3),Felony Petit Theft,2014-11-24,118.0,8.0
4,6,1,Male,37,25 - 45,African-American,0,0,0,2,(M1),Battery,2013-12-24,0.0,32.0


In [124]:
training_Target.head()

,index,Unnamed: 0,custody_count,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc,bust_date,jail_time,custody_sum
0,2620,3571,2,0.365072,19,0.427804,0.401425,0,0,0,0,0.375116,0.325048,0.374970,5.0,249.0
1,4779,6529,4,0.252126,34,0.346495,0.293271,0,0,0,0,0.362039,0.415094,0.333333,2.0,71.0
2,3133,4318,1,0.365072,34,0.346495,0.401425,0,0,0,0,0.375116,0.200000,0.250083,2.0,2.0
3,7748,10472,1,0.365072,36,0.346495,0.401425,0,0,1,16,0.375116,0.111188,0.090920,1.0,1.0
4,7412,10039,1,0.252126,27,0.346495,0.401425,0,0,0,0,0.362039,0.444444,0.533333,1.0,1.0


In [125]:
training_OneHot.head()

,sex_1,sex_2,sex_-1,age_cat_1,age_cat_2,age_cat_3,age_cat_-1,race_1,race_2,race_3,...,index,Unnamed: 0,custody_count,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,custody_sum
0,1,0,0,1,0,0,0,1,0,0,...,2620,3571,2,19,0,0,0,0,5.0,249.0
1,0,1,0,0,1,0,0,0,1,0,...,4779,6529,4,34,0,0,0,0,2.0,71.0
2,1,0,0,0,1,0,0,1,0,0,...,3133,4318,1,34,0,0,0,0,2.0,2.0
3,1,0,0,0,1,0,0,1,0,0,...,7748,10472,1,36,0,0,1,16,1.0,1.0
4,0,1,0,0,1,0,0,1,0,0,...,7412,10039,1,27,0,0,0,0,1.0,1.0


In [134]:
training_LeaveOneOut.head()

,index,Unnamed: 0,custody_count,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc,bust_date,jail_time,custody_sum
0,2620,3571,2,0.365130,19,0.428060,0.401527,0,0,0,0,0.375204,0.325670,0.428571,5.0,249.0
1,4779,6529,4,0.251672,34,0.346350,0.293017,0,0,0,0,0.360915,0.403846,0.285714,2.0,71.0
2,3133,4318,1,0.365130,34,0.346572,0.401527,0,0,0,0,0.375204,0.208333,0.285714,2.0,2.0
3,7748,10472,1,0.365130,36,0.346572,0.401527,0,0,1,16,0.375204,0.125000,0.100000,1.0,1.0
4,7412,10039,1,0.252280,27,0.346572,0.401527,0,0,0,0,0.362676,0.457143,0.571429,1.0,1.0


# Testowanie na klasyfikatorach

W następnym kroku postaramy się sprawdzić jak kodowanie może wpłynąć na skuteczność klasyfikatora. W celu uniknięcia overfittingu przy kodowaniu zmiennych na początku nie użyjemy kroswalidacji do oceniania wyników klasyfikacji. Tak prezentują się wyniki.

`Target Encoding`

In [194]:

np.random.seed(42)

classifier = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=3)

classifier = classifier.fit(training_Target, y_train)

classifier.score(testing_Target, y_test)

        

0.7170288024254674

`OneHot Encoding`

In [195]:

np.random.seed(42)
classifier = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=3)

classifier = classifier.fit(training_OneHot, y_train)

classifier.score(testing_OneHot, y_test)

0.6437594744820616

`LeaveOneOut Encoding`

In [196]:
np.random.seed(42)
classifier = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=3)

classifier = classifier.fit(training_LeaveOneOut, y_train)

classifier.score(testing_LeaveOneOut, y_test)

0.6432541687721072

`Hashing encoding`

In [197]:
np.random.seed(42)
classifier = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=3)

classifier = classifier.fit(training_Hashing, y_train)

classifier.score(testing_Hashing, y_test)

0.7124810510358767

`Usunięcie zmiennych kategorycznych`

In [198]:
X_train_without = X_train.drop(["sex","age_cat","race","c_charge_degree", "c_charge_desc","bust_date"], axis = 1)
X_test_without = X_test.drop(["sex","age_cat","race","c_charge_degree", "c_charge_desc","bust_date"], axis = 1)

In [222]:
np.random.seed(4)
classifier = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=3)

classifier = classifier.fit(X_train_without, y_train)

classifier.score(X_test_without, y_test)

0.7483577564426478

Wyniki są dość zaskakujące, ponieważ najlepszy wyniki uzyskała ramka danych z usuniętymi kolumnami. W celu lepszej analizy skuteczność klasyfiaktowrów będziemy mierzyć przy pomocy kroswalidacji. Może to wpłynąć na przeuczenie się w przypadku dopasowywanie kodowania na całej ramce, a nie tylko na częśći uczącej. Dość dobrze poradziło sobie kodowanie `Target`, jednak tego typu kodowania mogą być kosztowne obliczeniowo.

# Testowanie przy użyciu kroswalidacji.

In [214]:

X = X[:3000]
y = y[:3000]



enc_Target = ce.TargetEncoder().fit(X, y)
enc_OneHot = ce.OneHotEncoder().fit(X, y)
enc_LeaveOneOut = ce.LeaveOneOutEncoder().fit(X, y)
enc_Hashing = ce.HashingEncoder().fit(X, y)

training_Target = enc_Target.transform(X, y)
training_OneHot = enc_OneHot.transform(X)
training_LeaveOneOut = enc_LeaveOneOut.transform(X, y)
training_Hashing = enc_Hashing.transform(X)

X = X.drop(["sex","age_cat","race","c_charge_degree", "c_charge_desc","bust_date"], axis = 1)






`LogisticRegression`

In [215]:
np.random.seed(1)
from sklearn.model_selection import cross_val_score
clf = LogisticRegression()
scores = cross_val_score(clf, training_Target, y, cv=5)
print("Accuracy Target: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = LogisticRegression()
scores = cross_val_score(clf, training_OneHot, y, cv=5)
print("Accuracy OneHot: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = LogisticRegression()
scores = cross_val_score(clf, training_LeaveOneOut, y, cv=5)
print("Accuracy LeaveOneOut: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = LogisticRegression()
scores = cross_val_score(clf, training_Hashing, y, cv=5)
print("Accuracy Hashing: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = LogisticRegression()
scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy Without Variables: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy Target: 0.79 (+/- 0.03)
Accuracy OneHot: 0.73 (+/- 0.04)
Accuracy LeaveOneOut: 0.74 (+/- 0.03)
Accuracy Hashing: 0.74 (+/- 0.03)
Accuracy Without Variables: 0.74 (+/- 0.03)


`SVC`

In [216]:
np.random.seed(1)
from sklearn.model_selection import cross_val_score
clf = SVC()
scores = cross_val_score(clf, training_Target, y, cv=5)
print("Accuracy Target: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = SVC()
scores = cross_val_score(clf, training_OneHot, y, cv=5)
print("Accuracy OneHot: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = SVC()
scores = cross_val_score(clf, training_LeaveOneOut, y, cv=5)
print("Accuracy LeaveOneOut: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = SVC()
scores = cross_val_score(clf, training_Hashing, y, cv=5)
print("Accuracy Hashing: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = SVC()
scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy Without Variables: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy Target: 0.66 (+/- 0.00)
Accuracy OneHot: 0.61 (+/- 0.09)
Accuracy LeaveOneOut: 0.66 (+/- 0.00)
Accuracy Hashing: 0.66 (+/- 0.00)
Accuracy Without Variables: 0.66 (+/- 0.00)


`RandomForestClassifier`

In [217]:
np.random.seed(1)
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier()
scores = cross_val_score(clf, training_Target, y, cv=5)
print("Accuracy Target: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = RandomForestClassifier()
scores = cross_val_score(clf, training_OneHot, y, cv=5)
print("Accuracy OneHot: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = RandomForestClassifier()
scores = cross_val_score(clf, training_LeaveOneOut, y, cv=5)
print("Accuracy LeaveOneOut: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = RandomForestClassifier()
scores = cross_val_score(clf, training_Hashing, y, cv=5)
print("Accuracy Hashing: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = RandomForestClassifier()
scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy Without Variables: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy Target: 0.77 (+/- 0.03)
Accuracy OneHot: 0.70 (+/- 0.04)
Accuracy LeaveOneOut: 1.00 (+/- 0.00)
Accuracy Hashing: 0.71 (+/- 0.06)
Accuracy Without Variables: 0.71 (+/- 0.06)


`AdaBoostClassifier`

In [220]:
np.random.seed(1)
from sklearn.model_selection import cross_val_score
clf = AdaBoostClassifier()
scores = cross_val_score(clf, training_Target, y, cv=5)
print("Accuracy Target: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = AdaBoostClassifier()
scores = cross_val_score(clf, training_OneHot, y, cv=5)
print("Accuracy OneHot: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = AdaBoostClassifier()
scores = cross_val_score(clf, training_LeaveOneOut, y, cv=5)
print("Accuracy LeaveOneOut: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = AdaBoostClassifier()
scores = cross_val_score(clf, training_Hashing, y, cv=5)
print("Accuracy Hashing: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = AdaBoostClassifier()
scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy Without Variables: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy Target: 0.70 (+/- 0.20)
Accuracy OneHot: 0.64 (+/- 0.30)
Accuracy LeaveOneOut: 1.00 (+/- 0.00)
Accuracy Hashing: 0.62 (+/- 0.28)
Accuracy Without Variables: 0.64 (+/- 0.29)


`GaussianProcessClassifier`

In [221]:
np.random.seed(1)
from sklearn.model_selection import cross_val_score
clf = GaussianProcessClassifier()
scores = cross_val_score(clf, training_Target, y, cv=5)
print("Accuracy Target: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = GaussianProcessClassifier()
scores = cross_val_score(clf, training_OneHot, y, cv=5)
print("Accuracy OneHot: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = GaussianProcessClassifier()
scores = cross_val_score(clf, training_LeaveOneOut, y, cv=5)
print("Accuracy LeaveOneOut: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = GaussianProcessClassifier()
scores = cross_val_score(clf, training_Hashing, y, cv=5)
print("Accuracy Hashing: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = GaussianProcessClassifier()
scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy Without Variables: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy Target: 0.61 (+/- 0.09)
Accuracy OneHot: 0.61 (+/- 0.09)
Accuracy LeaveOneOut: 0.61 (+/- 0.09)
Accuracy Hashing: 0.61 (+/- 0.09)
Accuracy Without Variables: 0.61 (+/- 0.09)


# Wnioski

Tutaj w pierwszym klasyfikatorze widać znaczącą przewagę `Target` encodingu nawet nad `LeaveOneOut` encodingiem co jest dość zaskakujące, gdyż są to bardzo podobne kodowania. Pózniej kilka razy z rzędu `LeaveOneOut` osiąga zaskakujące `1.00 Accuracy`, co niestety raczej jest spowodowane overfittingiem kodowania, o którym wspominałem wcześniej. W celu pozbycia się aż tak dużego narzuty należałoby dopasowywać kodowanie tylko na danych uczących, co robiłem na początku raportu. Proste kodowanie poradziły sobie podobnie do ramki danych bez zmiennych. Prawdopodobnie dla lepszej oceny powinno się przetestować te kodowania na różnych ramkach danych, z różnymi klasyfikatorami i z różnymi miarami (oraz dopasowywac kodowanie tylko na zbiorze uczący, o czym wspominałem wcześniej), wtedy możnaby było wyciągać bardziej konkretne wnioski. Skuteczność kodowania zależy od znaczenia kodowanych zmiennych kategorycznych i czasami nawet lepiej jest po prostu usnunąć małow wnoszące zmienne. Jeśli chodzi o wystąpienie nowych poziomów zmiennej w danych testowych / nowych danych, to część kodowań lub pakiet `vtreat` z `R`-a robią to automatycznie przy transformacji danych. Może to być zrobione przez przedstawianie nowych zmiennych jako 'no level', czyli tworzenie dodatkowej wartości dla wszystkich nowych leveli. Inną opcją jest ważenie nowych poziomów proporcjonalnie do tych już znanych lub traktowanie nowych poziomów jako niepewności wśród rzadkich poziomów.